In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp "/content/drive/MyDrive/David_Copperfield.txt" /content/

In [3]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124


In [4]:
!pip cache purge

Files removed: 76


In [5]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" --no-deps

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ilxyfvxs/unsloth_d778054b41ea496dba0131b1cce39d20
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ilxyfvxs/unsloth_d778054b41ea496dba0131b1cce39d20
  Resolved https://github.com/unslothai/unsloth.git to commit 3340eaa41ce58619daf79d2783e49a45f8553a61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!pip install torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.5/768.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 103.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.30 requires torch==2.7.0, but you have torch 2.6.0+cu124 which is incompatible.
unsloth-zoo 2025.5.11 requires protobuf<4.0.0, but you have protobuf 6.31.1 which is incompatible.


In [7]:
!pip install --upgrade protobuf>=5.26.1 trl==0.8.6

In [8]:
from huggingface_hub import login
login()


In [9]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import LlamaTokenizer, LlamaForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
from transformers import LlamaTokenizerFast, LlamaForCausalLM
from unsloth import FastLanguageModel
from trl import SFTTrainer

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

<ipython-input-9-d70cc627cdf5>:7: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

# tokenizer = LlamaTokenizerFast.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token

# model = LlamaForCausalLM.from_pretrained(
#     model_name,
#     load_in_8bit=True,
#     device_map="auto"
# )
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 1024,
    load_in_8bit = True,
    load_in_4bit = False
)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,  # Recommended 0 for Unsloth
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)


==((====))==  Unsloth 2025.5.10: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: Making `model.base_model.model.model` require gradients


In [11]:
from datasets import Dataset

with open("/content/drive/MyDrive/David_Copperfield.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Remove empty lines and strip
lines = [line.strip() for line in lines if line.strip()]

# Create a Dataset from a list of dicts
dataset = Dataset.from_dict({"text": lines})


In [12]:
# data_path = "/content/David_Copperfield.txt"
# dataset = load_dataset("text", data_files={"train": data_path})

In [13]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding = "max_length",
        truncation = True,
        max_length = 512,
        return_tensors = "pt",
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/7308 [00:00<?, ? examples/s]

In [14]:
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM,
# )

# model = get_peft_model(model, lora_config)

In [15]:
training_args = TrainingArguments(
    output_dir="./llama-lora-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=3e-4,
    fp16=True,
    save_steps=500,
    save_total_limit=2,
    logging_steps=10,
    eval_strategy="no",
    warmup_steps=100,
    report_to=[],#for disabling wandb
)

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
# trainer = FastLanguageModel.get_trainer(
#     model = model,
#     args = training_args,
#     train_dataset = tokenized_dataset,
# )

In [17]:
# slothai.patch()

In [18]:
trainer = SFTTrainer(
    model = model,
    args = training_args,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 512,
    tokenizer = tokenizer,
    packing = False
)


Map:   0%|          | 0/7308 [00:00<?, ? examples/s]

In [19]:
try:
    trainer.train()
except Exception as e:
    print(f"Training error: {e}")
    print("Trying fallback without gradient checkpointing...")
    model.gradient_checkpointing_disable()
    trainer.train()

# 10. Save Model
model.save_pretrained("llama-3-lora-finetuned")
tokenizer.save_pretrained("llama-3-lora-finetuned")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,308 | Num Epochs = 2 | Total steps = 1,828
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 9,175,040/3,221,924,864 (0.28% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.966400
20,3.893300
30,3.726600
40,3.454200
50,3.401700
60,3.090000
70,3.048400
80,3.092300
90,3.105400
100,3.047200


Unsloth: Will smartly offload gradients to save VRAM!


('llama-3-lora-finetuned/tokenizer_config.json',
 'llama-3-lora-finetuned/special_tokens_map.json',
 'llama-3-lora-finetuned/chat_template.jinja',
 'llama-3-lora-finetuned/tokenizer.json')